In [1]:
# Notebook to play around with openmm and integrate how to implement constant salt simulations in the semi-grand
# canonical ensemble.

In [2]:
import numpy as np
from numpy import random
from simtk import openmm, unit
from simtk.openmm import app
import sys
from sys import stdout
# CONSTANTS
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

In [3]:
# Loading a box of water and initialing a protoms simulation
pdb = app.PDBFile('waterbox.pdb')
forcefield = app.ForceField('tip3p.xml')
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
system.addForce(openmm.MonteCarloBarostat(1*unit.atmospheres, 300*unit.kelvin, 25))
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [4]:
# Run a simulation
print "Minimizing energy..."
simulation.minimizeEnergy(maxIterations=25)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

print "Running simulation..."
simulation.reporters.append(app.PDBReporter('output.pdb',50))
simulation.reporters.append(app.StateDataReporter(stdout,50,step=True,potentialEnergy=True,volume=True))
simulation.step(100)

Minimizing energy...
Running simulation...
#"Step","Potential Energy (kJ/mole)","Box Volume (nm^3)"
50,-18874.744203,15.6044457135
100,-18824.3502152,15.5902731306


In [5]:
# Trying to understand the force class:
forces = []
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    forces.append(force)
    print force.__class__.__name__

#force_type = ""
#force_index = 0
#while force_type != system.getForce(force_index)__class__.__name__
#    non_bonded_force    
    
    
force_classes_to_update = ['NonbondedForce']
forces_to_update = list()
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if force.__class__.__name__ in force_classes_to_update:
        forces_to_update.append(force)    
print forces_to_update

forces_to_update[0].getParticleParameters(119)

HarmonicBondForce
HarmonicAngleForce
NonbondedForce
CMMotionRemover
MonteCarloBarostat
[<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x10b0a6180> >]


[Quantity(value=0.417, unit=elementary charge),
 Quantity(value=1.0, unit=nanometer),
 Quantity(value=0.0, unit=kilojoule/mole)]

In [6]:
# Seeing how to update forces
force = forces[2]
print force.getParticleParameters(100)
force.setParticleParameters(100,charge=1,sigma=0.1,epsilon=1)
print force.getParticleParameters(100)

[Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
[Quantity(value=1.0, unit=elementary charge), Quantity(value=0.1, unit=nanometer), Quantity(value=1.0, unit=kilojoule/mole)]


In [7]:
force_class_to_update = ['NonbondedForce']
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if force.__class__.__name__ in force_class_to_update:
        forces_to_update = force   
#print forces_to_update

forces_to_update.getParticleParameters(119)
# Find water molecules. 
# Loop okay for now, but better to loop once and update the list per exchange

water_identities =  np.zeros(pdb.topology.residues.im_self.getNumResidues())  
water_residue_names=('WAT', 'HOH', 'TP4', 'TP5', 'T4E')
water_residues = list()
k=0
for residue in pdb.topology.residues():
    if residue.name in water_residue_names:
        water_residues.append(residue)
        water_identities[k] = 1
    k += 1    

#print water_identities
 
print water_residues[0].atoms
    
water_identities =  np.zeros(pdb.topology.residues.im_self.getNumResidues())        
#print pdb.topology.residues()

        
# Get current energy
state = simulation.context.getState(getEnergy=True)        
pot_energy_old = state.getPotentialEnergy()

print "old potential energy = ", pot_energy_old

# Select random water molecules
# Here could add the option to create multiple pairs of waters options for configurational biasing
indices = np.random.random_integers(low=0,high=len(water_residues),size=2)    
print "Using these water residues", indices

num_wat_atoms = 3
cation_parameters = [{'charge': 0.0, 'sigma': 0.0, 'epsilon': 0.0}]*num_wat_atoms
cation_parameters[0] = {'charge': 1.0, 'sigma': 0.4477657, 'epsilon': 0.148912744}
#print cation_parameters

target_force = cation_parameters[0]
print target_force["charge"]


<bound method Residue.atoms of <Residue 0 (HOH) of chain 0>>
old potential energy =  -18824.3502152 kJ/mol
Using these water residues [149 144]
1.0


In [8]:
w=water_residues[1]
w.atoms

<bound method Residue.atoms of <Residue 1 (HOH) of chain 0>>

In [9]:
res =water_residues[2]
indices = [res.index for res in water_residues]
#print indices

stateMatrix = np.zeros((3,pdb.topology.residues.im_self.getNumResidues()))
water_indices = [res.index for res in water_residues]
stateMatrix[0,water_indices] = 1
#print stateMatrix

print "here are the waters",np.where(stateMatrix[0] == 1)[0]
#np.random.choice

np.random.choice(a=np.where(stateMatrix[0] == 1)[0],size=2)

here are the waters [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 24

array([442, 255])

In [10]:
print water_residues[1]
# convert to cation:
water = water_residues[indices[0]]
molecule = [atom for atom in water.atoms()]
print molecule
ion_index=0
for atom in molecule:
    current_force = forces_to_update.getParticleParameters(atom.index)
    print ("\n for atom %i, current force parameters are" % atom.index), current_force
    target_force = cation_parameters[ion_index]
    print "\n will attempt change to", target_force
    forces_to_update.setParticleParameters(atom.index,charge=target_force["charge"],sigma=target_force["sigma"],epsilon=target_force["epsilon"])
    ion_index += 1

# The force gets updated, which is why I may need to do a "deep copy"    
forces_to_update.updateParametersInContext(simulation.context)    
state = simulation.context.getState(getEnergy=True)    
pot_energy_new = state.getPotentialEnergy()
print pot_energy_new

<Residue 1 (HOH) of chain 0>
[<Atom 0 (O) of chain 0 residue 0 (HOH)>, <Atom 1 (H1) of chain 0 residue 0 (HOH)>, <Atom 2 (H2) of chain 0 residue 0 (HOH)>]

 for atom 0, current force parameters are [Quantity(value=-0.834, unit=elementary charge), Quantity(value=0.3150752406575124, unit=nanometer), Quantity(value=0.635968, unit=kilojoule/mole)]

 will attempt change to {'epsilon': 0.148912744, 'charge': 1.0, 'sigma': 0.4477657}

 for atom 1, current force parameters are [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]

 will attempt change to {'epsilon': 0.0, 'charge': 0.0, 'sigma': 0.0}

 for atom 2, current force parameters are [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]

 will attempt change to {'epsilon': 0.0, 'charge': 0.0, 'sigma': 0.0}
-18684.963209 kJ/mol


In [11]:
water = water_residues[1]
atoms = [atom for atom in water.atoms()]
print atoms[0].index
print atoms

3
[<Atom 3 (O) of chain 0 residue 1 (HOH)>, <Atom 4 (H1) of chain 0 residue 1 (HOH)>, <Atom 5 (H2) of chain 0 residue 1 (HOH)>]


In [12]:
sys.path.append("/Users/rossg/Work/semigrand-counterions")
import constant-ions

SyntaxError: invalid syntax (<ipython-input-12-029315c248b3>, line 2)

In [ ]:
force

In [ ]:
water_residies = identifyResidues(pdb.topology,residue_names = ('WAT', 'HOH', 'TP4', 'TP5', 'T4E'))
print len(water_residues)